In [26]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [27]:
data=pd.read_csv("OnlineRetail.csv",encoding='unicode_escape')
data.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom

In [28]:
data.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [29]:
data['Description' ]=data['Description'].str. strip()
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True)
data['InvoiceNo' ]=data['InvoiceNo'].astype('str')
data = data[~data['InvoiceNo'].str.contains('C')]

In [30]:
basket_France = (data[data['Country' ] == "France"]
.groupby(['InvoiceNo', 'Description' ]) [ 'Quantity' ]
.sum().unstack().reset_index().fillna(0)
.set_index('InvoiceNo'))

basket_UK = (data[data['Country' ] == "United Kingdom"]
.groupby(['InvoiceNo', 'Description' ]) [ 'Quantity' ]
.sum().unstack().reset_index().fillna(0)
.set_index('InvoiceNo'))

basket_Por = (data[data['Country' ] == "Portugal"]
.groupby(['InvoiceNo', 'Description' ]) ['Quantity' ]
.sum().unstack().reset_index().fillna(0)
.set_index('InvoiceNo'))

basket_Sweden = (data[data['Country' ] == "Sweden" ]
.groupby(['InvoiceNo', 'Description' ]) ['Quantity' ]
.sum().unstack().reset_index().fillna(0)
.set_index('InvoiceNo'))

In [31]:
def hot_encode(X): #Use the correct argument name here
    if X <= 0:#Access the argument using X
        return 0
        
    if X > 0:#Access the argument using X
        return 1


basket_encoded = basket_France.map(hot_encode)
basket_France = basket_encoded

basket_encoded = basket_UK.map(hot_encode)
basket_UK = basket_encoded

basket_encoded = basket_Por.map(hot_encode)
basket_Por = basket_encoded

basket_encoded = basket_Sweden.map(hot_encode)
basket_Sweden = basket_encoded


In [32]:

frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True)
rules = association_rules(frq_items, metric = "lift", min_threshold = 1)
rules = rules. sort_values(['confidence', 'lift' ], ascending = [False, False])
print(rules.head())

                                           antecedents  \
44                        (JUMBO BAG WOODLAND ANIMALS)   
260  (PLASTERS IN TIN CIRCUS PARADE, RED TOADSTOOL ...   
272  (PLASTERS IN TIN WOODLAND ANIMALS, RED TOADSTO...   
300  (SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...   
302  (SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...   

                         consequents  antecedent support  consequent support  \
44                         (POSTAGE)            0.076531            0.765306   
260                        (POSTAGE)            0.051020            0.765306   
272                        (POSTAGE)            0.053571            0.765306   
300  (SET/6 RED SPOTTY PAPER PLATES)            0.102041            0.127551   
302    (SET/6 RED SPOTTY PAPER CUPS)            0.102041            0.137755   

      support  confidence      lift  leverage  conviction  zhangs_metric  
44   0.076531       1.000  1.306667  0.017961         inf       0.254144  
260  0.051020       

In [ ]:
import warnings
warnings.filterwarnings('ignore')